# Advanced Composition

Learn advanced patterns for composing complex prompts from modular pieces.

In [1]:
import json

from t_prompts import prompt

## Building Prompt Libraries

Create reusable prompt components that can be composed together.

In [2]:
# Create a library of system prompts
SYSTEM_PROMPTS = {
    'helpful': prompt(t"{'You are a helpful assistant.':system}"),
    'concise': prompt(t"{'Be concise and direct.':system}"),
    'polite': prompt(t"{'Always be polite and respectful.':system}")
}

# Use them
query = "What is Python?"
full = prompt(t"{SYSTEM_PROMPTS['helpful']:sys}\n\nUser: {query:q}")
print(str(full))

You are a helpful assistant.

User: What is Python?


## Multi-Level Nesting

Build deeply nested structures for complex prompts.

In [3]:
# Level 1: Basic facts
name = "Alice"
age = "30"
person = prompt(t"Name: {name:n}, Age: {age:a}")

# Level 2: Context
city = "Paris"
context = prompt(t"{person:person} lives in {city:c}")

# Level 3: Full prompt
task = "Write a short bio"
full = prompt(t"Task: {task:t}\nContext: {context:ctx}")

print(str(full))
print("\nNavigate nested structure:")
print(f"  Name: {full['ctx']['person']['n'].value}")
print(f"  Age: {full['ctx']['person']['a'].value}")
print(f"  City: {full['ctx']['c'].value}")

Task: Write a short bio
Context: Name: Alice, Age: 30 lives in Paris

Navigate nested structure:
  Name: Alice
  Age: 30
  City: Paris


## Conditional Composition

Build prompts conditionally based on runtime data.

In [4]:
def build_prompt_with_context(query, include_examples=False, include_constraints=False):
    """Build a prompt with optional sections."""
    parts = []

    # Always include the query
    parts.append(prompt(t"Query: {query:q}"))

    # Optionally include examples
    if include_examples:
        examples = "Example: 2+2=4"
        parts.append(prompt(t"\n{examples:ex}"))

    # Optionally include constraints
    if include_constraints:
        constraints = "Keep response under 50 words"
        parts.append(prompt(t"\n{constraints:constraints}"))

    # Combine all parts into a list and interpolate
    return prompt(t"{parts:parts:sep=}")

# Try different combinations
p1 = build_prompt_with_context("What is 5+3?")
print("Minimal:")
print(str(p1))

p2 = build_prompt_with_context("What is 5+3?", include_examples=True, include_constraints=True)
print("\nWith everything:")
print(str(p2))

Minimal:
Query: What is 5+3?

With everything:
Query: What is 5+3?Example: 2+2=4Keep response under 50 words


## Prompt Templates with Slots

Create templates that can be filled in with specific values.

In [5]:
def create_translation_prompt(text, target_lang, include_context=True):
    """Template for translation prompts."""
    task = f"Translate to {target_lang}"
    p_task = prompt(t"{task:task}")
    p_text = prompt(t"Text: {text:text}")

    if include_context:
        context = "Maintain the original tone and style."
        p_context = prompt(t"Context: {context:ctx}")
        return prompt(t"{p_task:t}\n{p_context:c}\n{p_text:txt}")
    else:
        return prompt(t"{p_task:t}\n{p_text:txt}")

# Use the template
p = create_translation_prompt("Hello, world!", "French")
print(str(p))
print("\nKeys:", list(p.keys()))

Translate to French
Context: Maintain the original tone and style.
Text: Hello, world!

Keys: ['t', 'c', 'txt']


## Combining Lists and Nesting

Use lists within nested prompts for maximum flexibility.

In [6]:
# Create a list of examples
examples = [
    prompt(t"{ex:ex}")
    for ex in ["Python is a language", "JavaScript runs in browsers", "Rust is memory-safe"]
]

# Wrap the examples in a section
examples_section = prompt(t"Examples:\n{examples:examples}")

# Create the full prompt with the nested section
instruction = "Based on these examples, describe programming languages briefly."
full = prompt(t"{instruction:inst}\n\n{examples_section:ex_section}")

print(str(full))
print("\nAccess nested list:")
examples_node = full['ex_section']['examples']
print(f"Number of examples: {len(examples_node)}")
print(f"First example: {examples_node[0]['ex'].value}")

Based on these examples, describe programming languages briefly.

Examples:
Python is a language
JavaScript runs in browsers
Rust is memory-safe

Access nested list:
Number of examples: 3
First example: Python is a language


## Exporting Complex Structures

Use `to_provenance()` to export the full structure of complex nested prompts.

In [7]:
# Export the complex structure
provenance = full.to_provenance()
print(json.dumps(provenance, indent=2))

{
  "strings": [
    "",
    "\n\n",
    ""
  ],
  "nodes": [
    {
      "key": "inst",
      "expression": "instruction",
      "conversion": null,
      "format_spec": "inst",
      "render_hints": "",
      "index": 1,
      "value": "Based on these examples, describe programming languages briefly."
    },
    {
      "key": "ex_section",
      "expression": "examples_section",
      "conversion": null,
      "format_spec": "ex_section",
      "render_hints": "",
      "index": 3,
      "value": {
        "strings": [
          "Examples:\n",
          ""
        ],
        "nodes": [
          {
            "key": "examples",
            "expression": "examples",
            "conversion": null,
            "format_spec": "examples",
            "render_hints": "",
            "index": 1,
            "value": [
              {
                "strings": [
                  "",
                  ""
                ],
                "nodes": [
                  {
                   

## Pattern: Conversation History

Build conversation history from a list of messages.

In [8]:
def build_conversation(messages):
    """Build a conversation prompt from a list of (role, content) tuples."""
    message_prompts = []
    for i, (role, content) in enumerate(messages):
        msg = prompt(t"{role}: {content: msg_{i} }")
        message_prompts.append(msg)

    return prompt(t"{message_prompts:messages:sep=\n}")

# Build a conversation
conversation = build_conversation([
    ("User", "Hello!"),
    ("Assistant", "Hi! How can I help?"),
    ("User", "What is Python?"),
    ("Assistant", "Python is a programming language.")
])

print(str(conversation))
print("\nMessage count:", len(conversation['messages']))

User: Hello!
Assistant: Hi! How can I help?
User: What is Python?
Assistant: Python is a programming language.

Message count: 4
